In [1]:
%pip install newsapi-python

In [2]:
from newsapi import NewsApiClient
#my api KEY: 00b3f189be5749b8b82314b9d9d5c5ef
api = NewsApiClient(api_key='00b3f189be5749b8b82314b9d9d5c5ef')

In [54]:
from numpy import source
import pandas as pd
desList = []
sourceList = []
timeList = []
newsData = (api.get_everything(q='tesla'))
for article in newsData['articles']:
  sourceList.append(article['source']['name'])
  timeList.append(article['publishedAt'])
  desList.append(article['description'])


In [55]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sentimentList = []
def nltkSentiment(sentence):
  #NLTK anaylzer provides 4 datapoints in a dictionary --> compound datapoint is used; range is -1 to 1 where -1 being most negative and 1 being most positive
  anaylzer = SentimentIntensityAnalyzer()
  return anaylzer.polarity_scores(sentence)["compound"]

for i in desList:
  sentimentList.append(nltkSentiment(i));

newsDF = pd.DataFrame(list(zip(sourceList, desList, timeList, sentimentList)), columns=['Sources','Description','Publishing','Sentiments'])
display(newsDF)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Sources,Description,Publishing,Sentiments
0,The Verge,Tesla is reportedly close to building a new gi...,2023-09-14T17:44:11Z,0.0000
1,Engadget,The European Union (EU) is providing €352 mill...,2023-09-14T11:45:45Z,0.4019
2,The Verge,Hyundai will feature Tesla’s EV charging port ...,2023-10-05T12:05:00Z,0.1779
3,Engadget,Huawei\r\n is moving deeper into the electric ...,2023-09-25T20:09:18Z,0.4019
4,Engadget,NASA's pioneering OSIRIS-REx mission has succe...,2023-09-25T19:21:32Z,0.4939
...,...,...,...,...
95,Ars Technica,The site was going to make lithium iron phosph...,2023-09-25T21:30:53Z,0.0000
96,Autoblog,"Filed under:\n Detroit Auto Show,Jeep,Truck,SU...",2023-09-13T15:00:00Z,0.4215
97,Autoblog,Continue reading Parts of Tesla's Supercharger...,2023-09-18T19:15:00Z,-0.9022
98,Yahoo Entertainment,Investors considered the possibility that the ...,2023-10-06T13:36:55Z,0.7430
